In [4]:
import random

In [31]:
class Vignette ():
    idPhoto=[]
    tags = []
    orientation = ""
    def __init__(self,id,tags, ori):
        self.idPhoto=id
        self.tags = list(set(tags))
        self.orientation = ori
    def testScore(self,v2):
        tag1 = self.tags
        tag2 = v2.tags
        nbCommuns = 0
        nbDiff1 = 0
        nbDiff2 = 0
        scoreTotal=0
        for i in tag1:
            if(i in tag2):
                nbCommuns+=1
        nbDiff1 = len(tag1) - nbCommuns
        nbDiff2 = len(tag2) - nbCommuns
        scoreTotal+=min(nbCommuns,nbDiff1,nbDiff2)
        return scoreTotal
    def combineV(self,v2):
        if(v2.orientation != "V" or self.orientation != "V" or isinstance(self.idPhoto,list)):
            return False
        self.tags.extend(v2.tags)
        self.idPhoto = [self.idPhoto,v2.idPhoto]
        return True
    def isSingleV(self):
        if(self.orientation == "H" or isinstance(self.idPhoto,list)):
            return False
        return True
    def toString(self):
        return "Vignette : Id : "+str(self.idPhoto) + " Tags : " + str(self.tags) + " Orientation : "+self.orientation

class Presentation():
    listeV = []
    def __init__(self,arrayVignette=[]):
        self.listeV = arrayVignette
    def addVignette(self,v):
        self.listeV.append(v)
    def evaluate(self):
        """ Question 1.6
        Calcul du score de la presentation selon les regles defini par le projet"""
        if (len(self.listeV) <=1):
            return 0
        else:
            scoreTotal = 0
            for i in range(0, len(self.listeV)-1):
                tag1 = self.listeV[i].tags
                tag2 = self.listeV[i+1].tags
                nbCommuns = 0
                nbDiff1 = 0
                nbDiff2 = 0
                for i in tag1:
                    if(i in tag2):
                        nbCommuns+=1
                nbDiff1 = len(tag1) - nbCommuns
                nbDiff2 = len(tag2) - nbCommuns
                scoreTotal+=min(nbCommuns,nbDiff1,nbDiff2)
        return scoreTotal
    def copy(self):
        new = Presentation(copy.deepcopy(self.listeV))
        return new
    def swap(self,id1,id2):
        v1 = self.listeV[id1]
        v2 = self.listeV[id2]
        self.listeV[id1] = v2
        self.listeV[id2] = v1

In [6]:
def errorInPres(pres):
    ids = []
    for i in pres.listeV:
        if(i.isSingleV or i.orientation == "H"):
            if(i.idPhoto in ids):
                print("\nError with id : "+str(i.idPhoto))
                return True
            ids.append(i.idPhoto)
        else:
            if(i.idPhoto[0] in ids):
                print("\nError with id : "+str(i.idPhoto[0]))
            if(i.idPhoto[1] in ids):
                print("\nError with id : "+str(i.idPhoto[1]))
            if(i.idPhoto[0] in ids or i.idPhoto[1] in ids):
                return True
            ids.append(i.idPhoto[0])
            ids.append(i.idPhoto[1])
    return False

In [7]:
def saveInstance(instance,fileName):
    """
    Sauvegarde une instance dans un fichier texte
    """
    if(errorInPres(instance)):
        print("Error in presentation : Same id used twice")
    if(not ".txt"in fileName):
        fileName+=".txt"
    fichier = open(fileName,"w")
    fichier.write(str(len(instance.listeV))+"\n")
    for i in instance.listeV:
        if(i.orientation == "H" or i.isSingleV()):
            fichier.write(str(i.idPhoto)+"\n")
        else:
            fichier.write(str(i.idPhoto[0])+" "+str(i.idPhoto[1])+"\n")
    fichier.close()

In [8]:
def lireInstance(fileName,pourcent,verbose = False):
    """Exercice 1.2/1.3
    Prend un certain nombre de d'image dans une instance et en retourne deux tableaux:
    Un pour les images horizontales
    Un pour les verticales
    """
    instanceH = []
    instanceV = []
    fichier = open(fileName,"r")
    nb = int(fichier.readline())
    if(verbose):
        print(nb)
    cpt = 0
    line = fichier.readline()
    while(line != None and cpt < nb*pourcent):
        if(verbose):
            print(cpt)
            print(line)
        line = line.split(" ")
        ori = line[0]
        if(verbose):
            print("ori : "+ori)
        id = cpt
        nbTags = int(line[1])
        if(verbose):
            print("id : "+str(id))
        tags = []
        for i in range(2,len(line)):
            if("\n" in line[i]):
                tags.append(line[i][0:len(line[i])-1])
            else:
                tags.append(line[i])
            if(verbose):
                print(line[i])
        line = fichier.readline()
        if(ori == "H"):
            instanceH.append(Vignette(id,tags,ori))
        else:
            instanceV.append(Vignette(id,tags,ori))
        cpt+=1

    return instanceH,instanceV

In [9]:
def question13():
    """ Question 1.3
    lis les fichier de l'instance a_example.txt 
    Trie les photos dans deux tableau 
    Cree une presentation avec d abord les photos horizontales puis les verticales 2 a 2"""
    h,v = lireInstance("Instances/a_example.txt",100)
    print("Taille h: "+str(len(h)))
    print("Taille v: "+str(len(v)))
    pres = Presentation([])
    print("H")
    for i in h:
        pres.addVignette(i)
    print("V")
    for i in range(0,len(v),2):
        v = Vignette([v[i].idPhoto,v[i+1].idPhoto],v[i].tags+v[i+1].tags,"V")
        
        print("Add vignette "+v.toString())
        pres.addVignette(v)

    print("Presentation : Taille "+str(len( pres.listeV)))
    for i in pres.listeV:
        print(i.toString())
        
    print("Fin Pres")

    print("Score : "+str(pres.evaluate()))
    saveInstance(pres,"resultat.txt")

In [10]:
question13()

IndexError: list index out of range

In [11]:
def glouton(instanceH,instanceV,verbose = False):
    pres = Presentation([])
    if(random.uniform(0, 1)>0.5):
        iVignette = int(random.uniform(0,len(instanceH)))
        pres.addVignette(instanceH[iVignette])
        instanceH.remove(instanceH[iVignette])
    else:
        iVignette = int(random.uniform(0,len(instanceV)))
        iVignette2 = iVignette
        while(iVignette == iVignette2):
            iVignette2 = int(random.uniform(0,len(instanceV)))
        v = instanceV[iVignette]
        v2 = instanceV[iVignette2]
        
        v.combineV(v2)
        instanceV.remove(v)
        instanceV.remove(v2)
        pres.addVignette(v)
        
    while(len(instanceH)>0 or len(instanceV)>0):
        score = -1
        vType =""
        idV = -1
        idV2 = -1
        for i in range(0,len(instanceH)):
            test = pres.listeV[len(pres.listeV)-1].testScore(instanceH[i])
            if(test> score or idV == -1):
                score = test
                vType = "H"
                idV = i
        
        for i in range(0,len(instanceV)):
            for j in range(0,len(instanceV)):
                if(i!= j):
                    v = Vignette(instanceV[i].idPhoto,instanceV[i].tags,instanceV[i].orientation)
                    v.combineV(instanceV[j])
                    test = pres.listeV[len(pres.listeV)-1].testScore(v)
                    if(test> score or idV == -1):
                        score = test
                        vType = "V"
                        idV = i
                        idV2 = j
        
            
        if(vType == "H"):
            pres.addVignette(instanceH[idV])
            instanceH.remove(instanceH[idV])
        elif(vType == "V"):
            v = Vignette(instanceV[idV].idPhoto,instanceV[idV].tags,instanceV[idV].orientation)
            v.combineV(instanceV[idV2])
            if(verbose):
                print("\n####")
                print("id2"+str(idV2))
                print(str(instanceV[idV2].idPhoto))
                print(v.toString()+"\n")
            delV1 = instanceV[idV]
            delV2 = instanceV[idV2]
            instanceV.remove(delV1)
            instanceV.remove(delV2)
            pres.addVignette(v)
            if(verbose):
                print("####\n")
        elif(idV == -1):
            pres.addVignette(instanceV[0])
            instanceV.remove(instanceV[0])
    return pres
            
    

In [23]:
def gloutonOpti(instanceH,instanceV,nbARegarder,verbose = False):
    pres = Presentation([])
    if(random.uniform(0, 1)>0.5):
        iVignette = int(random.uniform(0,len(instanceH)))
        pres.addVignette(instanceH[iVignette])
        instanceH.remove(instanceH[iVignette])
    else:
        iVignette = int(random.uniform(0,len(instanceV)))
        iVignette2 = iVignette
        while(iVignette == iVignette2):
            iVignette2 = int(random.uniform(0,len(instanceV)))
        v = instanceV[iVignette]
        v2 = instanceV[iVignette2]
        
        v.combineV(v2)
        instanceV.remove(v)
        instanceV.remove(v2)
        pres.addVignette(v)
        
    while(len(instanceH)>0 or len(instanceV)>0):
        score = -1
        vType =""
        V = -1
        V2 = -1
        vignetteAEtudierH = random.sample(instanceH,min(len(instanceH),nbARegarder))
        for i in vignetteAEtudierH:
            test = pres.listeV[len(pres.listeV)-1].testScore(i)
            if(test> score or V == -1):
                score = test
                vType = "H"
                V = i
        
        vignetteAEtudierV = random.sample(instanceV,min(len(instanceV),nbARegarder))
        for i in vignetteAEtudierV:
            for j in vignetteAEtudierV:
                if(i!= j):
                    v = Vignette(i.idPhoto,i.tags,i.orientation)
                    v.combineV(j)
                    test = pres.listeV[len(pres.listeV)-1].testScore(v)
                    if(test> score or V == -1):
                        score = test
                        vType = "V"
                        V = i
                        V2 = j
        
            
        if(vType == "H"):
            pres.addVignette(V)
            instanceH.remove(V)
        elif(vType == "V"):
            v = Vignette(V.idPhoto,V.tags,V.orientation)
            v.combineV(V2)
            instanceV.remove(V)
            instanceV.remove(V2)
            pres.addVignette(v)
        elif(V == -1):
            pres.addVignette(instanceV[0])
            instanceV.remove(instanceV[0])
    return pres
            
    

In [21]:
h,v=lireInstance("Instances/c_memorable_moments.txt",0.1)
pre = glouton(h,v)
for i in pre.listeV:
    print(i.toString())
saveInstance(pre ,"resulat_glouton.txt")

Vignette : Id : [26, 36] Tags : ['tv1', 't9d1', 'tmr', 't2m', 'tg82', 't3m1', 'tz91', 't9x', 't0c', 'tjr1', 't3q', 'tjc', 'tg92', ['tv61', 'ttw1', 't242', 't261', 'tvh', 'trq', 'tqz1', 'tck', 'tlg1']] Orientation : V
Vignette : Id : 34 Tags : ['t882', 't9s1', 't9d1', 'tr2', 'tvc2', 'tbv', 'tz91', 't6t', 't7q1', 'tv32', 't351', 'tnr'] Orientation : H
Vignette : Id : 3 Tags : ['tg02', 'twd1', 't21', 't8b1', 't0c', 'tqp1', 't351', 'tl3', 'tqz1', 'tms'] Orientation : H
Vignette : Id : 16 Tags : ['tr2', 'tf01', 't8b1', 'tvh', 'td8'] Orientation : H
Vignette : Id : 62 Tags : ['tr5', 'tp32', 't8b1', 'tqb', 'tw52', 'tt', 't5f', 'txh'] Orientation : H
Vignette : Id : [0, 2] Tags : ['tjb1', 't8b1', 't17', 'tw52', 'tmz1', 'tg6', 't1l', ['tb6', 't4q', 't81', 't201', 'td7', 'tg11', 't001', 't652']] Orientation : V
Vignette : Id : [44, 6] Tags : ['t6f1', 't7k1', 't3m', 't4x', 't5l', 't412', 'tjb1', 'tgp1', 't1l1', 't0c', 't4r', 't13', 'th3', ['t9c1', 't502', 'trx', 't14', 'tr51', 't9s', 'tj62', 'tlb

In [28]:
h,v=lireInstance("Instances/c_memorable_moments.txt",0.1)
pre = gloutonOpti(h,v,100,True)
for i in pre.listeV:
    print(i.toString())
saveInstance(pre ,"resulat_gloutonOpti.txt")

Vignette : Id : 35 Tags : ['t3b1', 't34', 'td3', 'tvv', 'tvr', 't35', 'tm02', 't3k', 't0b1', 'tm81', 't6r1'] Orientation : H
Vignette : Id : [91, 20] Tags : ['tqh', 'tdn1', 'thh1', 'tnt', 'tbv', 't0g', 'tjt', 'tx52', 'tgt1', 'tw11', 't381', 't6r1', 'tm7', 'tlz1', 'txr', 'tvr', 't112', 'tvc2', 'tvs1', 'trh1', 't8q', 'tj21', 'tqc', 'tcl', 't3g', 't0g1', 'tvw', 't5x'] Orientation : V
Vignette : Id : [31, 40] Tags : ['t4c2', 'tvs1', 'tkc1', 't1c', 't73', 't0j', 't7q', 't6w', 't542', 'tsb2', 't4m1', 'tt01', 'tm7', 'tdn1', 'tx1', 'tv7', 'tfx', 'tbv', 'tr51', 't6h', 'tsf', 'td32', 'tq52', 'tlz1'] Orientation : V
Vignette : Id : [47, 67] Tags : ['trx1', 'tdn1', 'tvd1', 't29', 'td', 'tqz1', 'tq3', 't76', 'tzn', 'tsv', 'tk31', 'tvs1', 'tn02', 'tkc1', 'tnf', 't81', 't2m1', 'tpb2', 't5r1', 'tlb1', 't141', 'tjf1'] Orientation : V
Vignette : Id : [52, 49] Tags : ['t3b1', 't9s1', 't25', 'tv41', 'th8', 'tbc1', 'tv81', 'tsg', 'tcc', 'tf71', 'tcf', 'tzn', 'tb02', 'txw', 'tbn', 'tkc1', 'ttw1', 'tx6', 'tz

In [32]:
def scorePermutation(pres,idV,idNewV):
    newP = pres.copy()
    newP.swap(idV,idNewV)
    return newP.evaluate()

In [33]:
def descente_stochastiques(pres,it,nbVoisinnages,choixRandom):
    if(choixRandom):#On choisit une vignette au hasard pour tester différentes permutations
        cpt = 0
        while cpt < it:
            cpt+=1
            idV = int(random.uniform(0,len(pres.listeV)))
            bestEvalScore = pres.evaluate()
            idSwap1 = -1
            idSwap2 = -1
            for i in range(0,nbVoisinnages):
                idV2 = idV
                while(idV2 == idV or pres.listeV[idV].orientation != pres.listeV[idV2].orientation):
                    idV2 = int(random.uniform(0,len(pres.listeV)))
                newScore = scorePermutation(pres,idV,idV2)
                if(bestSwapScore < newScore):
                    bestSwapScore = newScore
                    idSwap1 = idV
                    idSwap2 = idV2
            if(idSwap1 != -1):
                pres.swap(idSwap1,idSwap2)
    else:#Toutes les vignettes pourront etre permutées
        for idV in range(0,len(pres.listeV)):
            bestEvalScore = pres.evaluate()
            idSwap1 = -1
            idSwap2 = -1
            for i in range(0,nbVoisinnages):
                idV2 = idV
                while(idV2 == idV or pres.listeV[idV].orientation != pres.listeV[idV2].orientation):
                    idV2 = int(random.uniform(0,len(pres.listeV)))
                newScore = scorePermutation(pres,idV,idV2)
                if(bestSwapScore < newScore):
                    bestSwapScore = newScore
                    idSwap1 = idV
                    idSwap2 = idV2
            if(idSwap1 != -1):
                pres.swap(idSwap1,idSwap2) 
         

SyntaxError: invalid syntax (<ipython-input-33-a7002f2e0634>, line 22)